In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1660 Ti


In [ ]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 1.9MB 14.6MB/s 
     |████████████████████████████████| 890kB 49.2MB/s 
     |████████████████████████████████| 3.2MB 45.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=1043c1a6d9f68e08173803fdc58e8b3266280361eefaba4c4428a8fcd48f6a1b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.2MB 21.8MB/s 


In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

model_name = "allenai/unifiedqa-t5-large" # you can specify the model size here "allenai/unifiedqa-t5-11b" 도 돌려줘
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def run_model(input_string, **generator_args):
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = model.generate(input_ids, **generator_args)
    return tokenizer.batch_decode(res, skip_special_tokens=True)

In [ ]:
import json
with open('toeic_bert_traindata.json') as input_json:
  data = json.load(input_json)

# Data is a dictionary contain over 3000 toeic question
# Let read the first question and familiar with format
data['1']

{'1': 'suffer',
 '2': 'suffers',
 '3': 'suffering',
 '4': 'suffered',
 'anwser': 'suffered',
 'question': 'The assets of Marble Faun Publishing Company ___ last quarter when one of their main local distributors went out of business.'}

In [ ]:
question_infors = []

for key, value in data.items():
    question_infors.append(value)

question_infors[0]

{'1': 'suffer',
 '2': 'suffers',
 '3': 'suffering',
 '4': 'suffered',
 'anwser': 'suffered',
 'question': 'The assets of Marble Faun Publishing Company ___ last quarter when one of their main local distributors went out of business.'}

In [ ]:
question_t5=[]
for i in question_infors:
  question=''.join(i['question'].replace('___','_'))+' \n '
  one='(a)'+''.join(i['1'])+' '
  two='(b)'+''.join(i['2'])+' '
  three='(c)'+''.join(i['3'])+' '
  four='(d)'+''.join(i['4'])
  question_t5.append(question+one+two+three+four)
  

In [ ]:
count = 0
for n,question in enumerate(question_t5[-100:]):
    anwser_predict = run_model(question)
    if anwser_predict == [question_infors[-100+n]['anwser']]:
        count+=1

num_questions = len(question_infors[-100:])
print(f'The model predict {round(count/num_questions,2) * 100} % of total {num_questions} questions')

The model predict 68.0 % of total 100 questions


In [ ]:
count = 0
for n,question in enumerate(question_t5):
    anwser_predict = run_model(question)
    if anwser_predict == [question_infors[n]['anwser']]:
        count+=1

num_questions = len(question_infors)
print(f'The model predict {round(count/num_questions,2) * 100} % of total {num_questions} questions')

The model predict 63.0 % of total 3625 questions
